# Google Calendar API Integration

Table of contents:
    

## OAuth 2.0 Setup

In [3]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [4]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [5]:
flow = InstalledAppFlow.from_client_secrets_file("credentials.json", scopes=scopes)


In [7]:
creds = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=871044571445-4mdju3plb9kvhlamm0q3l45p4em8qant.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=kZMgZhYojszU4ulhVsUuaErJ4Pz5dS&prompt=consent&access_type=offline
Enter the authorization code: 4/4AHp-7nMGdxpRyHtktFs-2knpEPos-a4Vwmv5FmV_103Gam6mOdnNTs


In [8]:
creds

 Creating a file using pickle to store the credentials so as to use it from time to time
 

In [9]:
import pickle


In [10]:
pickle.dump(creds, open("token.pkl", "wb"))

In [11]:
creds = pickle.load(open("token.pkl", "rb"))

In [12]:
service = build("calendar", "v3", credentials = creds)

## Get my calendars

In [13]:
result = service.calendarList().list().execute()

In [14]:
result


{'kind': 'calendar#calendarList',
 'etag': '"p32savlsgkrlem0g"',
 'nextSyncToken': 'CLiv15Cm6usCEhZjaGF2YW52aXNtaXRAZ21haWwuY29t',
 'items': [{'kind': 'calendar#calendarListEntry',
   'etag': '"1599732553037000"',
   'id': 'chavanvismit@gmail.com',
   'summary': 'chavanvismit@gmail.com',
   'timeZone': 'Asia/Kolkata',
   'colorId': '14',
   'backgroundColor': '#9fe1e7',
   'foregroundColor': '#000000',
   'selected': True,
   'accessRole': 'owner',
   'defaultReminders': [{'method': 'popup', 'minutes': 30}],
   'notificationSettings': {'notifications': [{'type': 'eventCreation',
      'method': 'email'},
     {'type': 'eventChange', 'method': 'email'},
     {'type': 'eventCancellation', 'method': 'email'},
     {'type': 'eventResponse', 'method': 'email'}]},
   'primary': True,
   'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}},
  {'kind': 'calendar#calendarListEntry',
   'etag': '"1599732552489000"',
   'id': 'addressbook#contacts@group.v.calendar.google.

## Get my calendar events

In [15]:
calendar_id = result['items'][0]['id']
show = service.events().list(calendarId = calendar_id).execute()

In [16]:
show

{'kind': 'calendar#events',
 'etag': '"p32savlsgkrlem0g"',
 'summary': 'chavanvismit@gmail.com',
 'updated': '2020-09-15T04:14:11.331Z',
 'timeZone': 'Asia/Kolkata',
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'nextSyncToken': 'CLiv15Cm6usCELiv15Cm6usCGAU=',
 'items': [{'kind': 'calendar#event',
   'etag': '"3181872687952000"',
   'id': '75h3idhn6cqmab9g65i3ab9k6ti30b9p6coj6b9mccomcohi6grj2e3374',
   'status': 'confirmed',
   'htmlLink': 'https://www.google.com/calendar/event?eid=NzVoM2lkaG42Y3FtYWI5ZzY1aTNhYjlrNnRpMzBiOXA2Y29qNmI5bWNjb21jb2hpNmdyajJlMzM3NCBjaGF2YW52aXNtaXRAbQ',
   'created': '2020-05-31T13:35:05.000Z',
   'updated': '2020-05-31T14:45:43.976Z',
   'summary': 'Discussion about Requirements and Initial Visualizations',
   'description': "Finish up Requirements document\nTake a look at Vismit's PR and take decisions to solve some issues",
   'creator': {'email': 'mohdabbas378@gmail.com'},
   'organizer': {'email': 'mohdabbas378@gmai

In [17]:
calendar_id = result['items'][2]['id']
show = service.events().list(calendarId = calendar_id).execute()

In [18]:
#show

## Generating a list of all holidays


In [19]:
#Creating a dataframe for holidays
import pandas as pd

holidays = pd.DataFrame(columns= ['Name','Date'])
dict= {}
Date = []
Name = []

#Converting the dict objects to input of the dataframe.
page_token = None
while True:
  events = service.events().list(calendarId= calendar_id, pageToken=page_token).execute()
  for event in events['items']:
        Name.append(event['summary'])
        dict.update(event['start'])
        Date.append(dict['date'])
        
    
    
  page_token = events.get('nextPageToken')
  if not page_token:
    break



In [20]:
holidays['Name'] = Name
holidays['Date'] = Date
holidays


,Name,Date
0,New Year's Day,2019-01-01
1,Guru Govind Singh Jayanti,2019-01-13
2,Lohri,2019-01-13
3,Makar Sankranti,2019-01-14
4,Pongal,2019-01-15
...,...,...
154,Bakr Id/Eid ul-Adha,2021-07-20
155,Vasant Panchami,2020-01-30
156,Ambedkar Jayanti,2020-04-14
157,Bakr Id/Eid ul-Adha,2020-08-01


In [21]:
holidays['Date']

0      2019-01-01
1      2019-01-13
2      2019-01-13
3      2019-01-14
4      2019-01-15
          ...    
154    2021-07-20
155    2020-01-30
156    2020-04-14
157    2020-08-01
158    2020-08-30
Name: Date, Length: 159, dtype: object

## Conversion of Date column from string to datetime

In [27]:
from datetime import datetime

holidays['Date'] = pd.to_datetime(holidays['Date'], format='%Y-%m-%d')

holidays

,Name,Date
0,New Year's Day,2019-01-01
1,Guru Govind Singh Jayanti,2019-01-13
2,Lohri,2019-01-13
3,Makar Sankranti,2019-01-14
4,Pongal,2019-01-15
...,...,...
154,Bakr Id/Eid ul-Adha,2021-07-20
155,Vasant Panchami,2020-01-30
156,Ambedkar Jayanti,2020-04-14
157,Bakr Id/Eid ul-Adha,2020-08-01


In [30]:
#Few search results as per Name or date to verify authenticity

holidays[holidays['Name']=='Dussehra']

,Name,Date
39,Dussehra,2019-10-08
82,Dussehra,2020-10-25
145,Dussehra,2021-10-15


In [31]:
holidays[holidays['Date']=='2019-10-29']

,Name,Date
45,Bhai Duj,2019-10-29


In [32]:
holidays[holidays['Date']=='2019-02-19']

,Name,Date
7,Shivaji Jayanti,2019-02-19
8,Guru Ravidas Jayanti,2019-02-19
